<a href="https://colab.research.google.com/github/m0zzarella/Fundus-Imaging/blob/main/blood_vessels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2 as cv
import numpy as np
from google.colab import drive
import glob
import os
!pip install image_similarity_measures
import image_similarity_measures
from image_similarity_measures.quality_metrics import rmse, psnr

     |████████████████████████████████| 19.3 MB 1.1 MB/s 
     |████████████████████████████████| 2.6 MB 37.8 MB/s 
  Created wheel for phasepack: filename=phasepack-1.5-py3-none-any.whl size=26983 sha256=e8d2911dc772382cdedb9caa1aba4d3f2ef0a4a0a3ec0a9db07a956cbc2e44d8
  Stored in directory: /root/.cache/pip/wheels/39/5c/b3/79292d7ac3f088e858805b5515b45cf5a365106397d936caf1
Successfully built phasepack


In [ ]:
from skimage import data, io, filters
from skimage.metrics import structural_similarity

In [ ]:
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/Fundus_Imaging/fundus_images/"

/content/drive/MyDrive/Fundus_Imaging/fundus_images


In [ ]:
path = "/content/drive/MyDrive/Fundus_Imaging/fundus_images/"
savepath = "/content/drive/MyDrive/Fundus_Imaging/Results/"
for file in os.listdir(path):
   #print(os.listdir(path))
   #print(file)
   img = cv.imread(file, cv.IMREAD_UNCHANGED)
   b,green_channel,r = cv.split(img)
   clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))   
   enhanced_green_channel = clahe.apply(green_channel)
   open1 = cv.morphologyEx(enhanced_green_channel, cv.MORPH_OPEN, cv.getStructuringElement(cv.MORPH_ELLIPSE,(5,5)), iterations = 1)
   close1 = cv.morphologyEx(open1, cv.MORPH_CLOSE, cv.getStructuringElement(cv.MORPH_ELLIPSE,(5,5)), iterations = 1)
   open2 = cv.morphologyEx(close1, cv.MORPH_OPEN, cv.getStructuringElement(cv.MORPH_ELLIPSE,(15,15)), iterations = 1)
   close2 = cv.morphologyEx(open2, cv.MORPH_CLOSE, cv.getStructuringElement(cv.MORPH_ELLIPSE,(15,15)), iterations = 1)
   open3 = cv.morphologyEx(close2, cv.MORPH_OPEN, cv.getStructuringElement(cv.MORPH_ELLIPSE,(30,30)), iterations = 1)
   close3 = cv.morphologyEx(open3, cv.MORPH_CLOSE, cv.getStructuringElement(cv.MORPH_ELLIPSE,(30,30)), iterations = 1)

   subt = cv.subtract(close3,enhanced_green_channel)
   img2 = clahe.apply(subt)
   ret,bin1 = cv.threshold(img2,15,255,cv.THRESH_BINARY)	

   mask = np.ones(img2.shape[:2], dtype="uint8") * 255	
   contours = cv.findContours(bin1.copy(),cv.RETR_LIST,cv.CHAIN_APPROX_SIMPLE)[0]

   #to remove the contours
   for j in contours:
	    if cv.contourArea(j) <= 200:
		     cv.drawContours(mask, [j], -1, 0, -1)			
   img3 = cv.bitwise_and(img2, img2, mask=mask)

   cont_rem = cv.erode(img3, cv.getStructuringElement(cv.MORPH_ELLIPSE,(3,3)), iterations=1)
   ret,blood_vessels = cv.threshold(cont_rem,11,255,cv.THRESH_BINARY)
   cv.imwrite(os.path.join(savepath,file), blood_vessels)

In [ ]:
%cd "/content/drive/MyDrive/Fundus_Imaging/Results"

In [ ]:
path1 = "/content/drive/MyDrive/Fundus_Imaging/Ground_Truth/"
path2 = "/content/drive/MyDrive/Fundus_Imaging/Results/"
rmse_arr = []
psnr_arr = []
ssim_arr = []
for file in os.listdir(path2):
   in_img2 = cv.imread(file)
   in_img1 = cv.imread(os.path.join(path1,file.split(".")[0]+'.png'))
   #print(in_img1.shape)
   #print(in_img2.shape)
   rmse_out = rmse(in_img1, in_img2)       #root mean square error
   rmse_arr.append(float(rmse_out)) 
   psnr_out = psnr(in_img1, in_img2)       #peak signal to noise ratio
   psnr_arr.append(float(psnr_out))
   (score, diff) = structural_similarity(cv.cvtColor(in_img2, cv.COLOR_BGR2GRAY), cv.cvtColor(in_img1, cv.COLOR_BGR2GRAY), full=True)
   diff = (diff * 255).astype("uint8")
   ssim_arr.append(float(score))           #structural similarity index
   
rmse_arr = np.array(rmse_arr)
psnr_arr = np.array(psnr_arr)
ssim_arr = np.array(score)
print('Average root mean square error: ' + str(np.average(rmse_arr)))
print('Average peak signal to noise ratio: ' + str(np.average(psnr_arr)))
print('Average structural similarity index values: ' + str(np.average(score)))

Average root mean square error: 0.012812103579441706
Average peak signal to noise ratio: 37.87098782126279
Average structural similarity index values: 0.8582882934168227
